# Training and deploying a tabular model using Vertex custom training job - Part 2

![Training pipeline](../images/custom-tabular.png)

In [ ]:
import os
import pprint
import pandas as pd
import tensorflow as tf
import time

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1beta1 import types

## Configure GCP settings


In [ ]:
PREFIX = 'jkwst1'

shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT = shell_output[0]
print("Project ID: ", PROJECT)

In [ ]:
REGION = 'us-central1'
STAGING_BUCKET = f'gs://{PREFIX}-bucket'

## Deploying a model to Vertex AI

### Initialize Vertex AI SDK

In [ ]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

### Retrieve pre-trained model artifacts

In [ ]:
pretrained_model_path = 'gs://workshop-datasets/models/taxi_classifier'

!gsutil ls {pretrained_model_path}

### Copy the pre-trained model artifacts to your staging bucket

In [ ]:
saved_model_path = f'{STAGING_BUCKET}/models/taxi_classifer'

!gsutil cp -r {pretrained_model_path}/* {saved_model_path}

### Inspect the model

In [ ]:
!saved_model_cli show --dir {saved_model_path} --tag_set serve --signature_def serving_default

### Upload the model

In [ ]:
display_name = f'{PREFIX} Chicago taxi classifier'
description = 'Chicago taxi tip TensorFlow classifier'
serving_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-4:latest'

model = vertex_ai.Model.upload(
    display_name=display_name,
    description=description,
    artifact_uri=saved_model_path,
    serving_container_image_uri=serving_image_uri
)

model.wait()

### Create an endpoint

In [ ]:
display_name = f'{PREFIX} Chicago taxi endpoint'

endpoint = vertex_ai.Endpoint.create(display_name=display_name)

### Deploy the Model

In [ ]:
deployed_model_display_name = f'{PREFIX}-taxi-v1'
traffic_percentage = 100
machine_type = 'n1-standard-4'

endpoint = model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        machine_type=machine_type,
        traffic_percentage=traffic_percentage)


## Invoking the deployed model using Vertex SDK

### Get the endpoint

In [ ]:
filter = f'display_name="{PREFIX} Chicago taxi endpoint"'

for endpoint_info in vertex_ai.Endpoint.list(filter=filter):
    print(endpoint_info)
    
endpoint = vertex_ai.Endpoint(endpoint_info.resource_name)

### Call the endpoint

In [ ]:
test_instances = [  
    
    {
        "dropoff_grid": ["POINT(-87.6 41.9)"],
        "euclidean": [2064.2696],
        "payment_type": ["Credit Card"],
        "pickup_grid": ["POINT(-87.6 41.9)"],
        "trip_miles": [1.37],
        "trip_day": [12],
        "trip_hour": [16],
        "trip_month": [2],
        "trip_day_of_week": [4],
        "trip_seconds": [555]
    }
]

predictions = endpoint.predict(instances=test_instances)
prob = tf.nn.sigmoid(predictions[0])
print('Probability of tip > 20%:', prob.numpy())

## Clean up

### Undeploy models

In [ ]:
endpoint.list_models()

In [ ]:
endpoint.undeploy_all()

### Delete endpoint

In [ ]:
endpoint.delete()

### Delete model

In [ ]:
model.delete()